# Train3 YOLO11L ko'krak bezi saratoni modeli tahlili


Ushbu notebook `train3/` papkasidagi YOLO11L modeli mashg'ulotlari natijalarini tahlil qiladi.
Quyidagi savollarga javob beramiz:

- O'qitish jarayonida yo'qotishlar va ko'rsatkichlar qanday o'zgargan?
- Qaysi epoch eng yuqori aniqlikka erishdi?
- Konfuzion matritsa va boshqa vizual natijalar nimani anglatadi?
- Model qanday parametrlar bilan o'qitilgan?

Notebookni to'liq ishga tushirish uchun Python muhitingizda `pandas`, `matplotlib`, `seaborn`, `pyyaml` va `Pillow` paketlari o'rnatilgan bo'lishi kerak.


In [ ]:
from pathlib import Path

import pandas as pd
import yaml
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Image, display

plt.style.use('seaborn-v0_8')
sns.set_theme(style='whitegrid', palette='crest')

results_dir = Path('train3')
if not results_dir.exists():
    raise FileNotFoundError('train3 katalogi topilmadi. Notebookni loyiha ildizidan ishga tushiring.')


## Epochlar kesimidagi natijalar

`results.csv` faylida har bir epoch uchun yo'qotishlar, aniqlik va qayta chaqirish kabi ko'rsatkichlar saqlangan.


In [ ]:
results_path = results_dir / 'results.csv'
df = pd.read_csv(results_path)

# Oxirgi ustunlardagi bo'sh qiymatlar tufayli paydo bo'ladigan qo'shimcha satrlarni tozalash
df = df.dropna(subset=['epoch'])
df['epoch'] = df['epoch'].astype(int)
df.head()


### Eng yaxshi epochlar va yakuniy ko'rsatkichlar

Har bir asosiy metrik uchun eng yuqori qiymat olingan epochlarni topamiz.


In [ ]:
metric_cols = {
    'metrics/precision(B)': 'Aniqlik (Precision)',
    'metrics/recall(B)': 'Qayta chaqirish (Recall)',
    'metrics/mAP50(B)': 'mAP@50',
    'metrics/mAP50-95(B)': 'mAP@50-95'
}

records = []
for col, label in metric_cols.items():
    idx = df[col].idxmax()
    row = df.loc[idx]
    records.append({
        "Ko'rsatkich": label,
        'Epoch': int(row['epoch']),
        'Qiymat': float(row[col])
    })

summary_df = pd.DataFrame(records)
summary_df


### Metrikalar dinamikasi

Aniqlik, qayta chaqirish va mAP ko'rsatkichlarining rivojlanishini chizamiz.


In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
for col, label in metric_cols.items():
    ax.plot(df['epoch'], df[col], label=label)
ax.set_xlabel('Epoch')
ax.set_ylabel('Qiymat')
ax.set_title("Validatsiya metrikalarining o'zgarishi")
ax.set_ylim(0, 1)
ax.legend(loc='best')
plt.tight_layout()
plt.show()


### Yo'qotish funksiyalari

Trening va validatsiya yo'qotishlari qanchalik tez kamayganini ko'rib chiqamiz.


In [ ]:
loss_pairs = [
    ('train/box_loss', 'val/box_loss', 'Box loss'),
    ('train/cls_loss', 'val/cls_loss', 'Class loss'),
    ('train/dfl_loss', 'val/dfl_loss', 'DFL loss')
]

fig, axes = plt.subplots(1, 3, figsize=(18, 4), sharex=True)
for ax, (train_col, val_col, title) in zip(axes, loss_pairs):
    ax.plot(df['epoch'], df[train_col], label='Train')
    ax.plot(df['epoch'], df[val_col], label='Validation')
    ax.set_title(title)
    ax.set_xlabel('Epoch')
    ax.set_ylabel('Loss')
    ax.legend()
plt.tight_layout()
plt.show()


## Vizual natijalar

YOLO 11L modeli chiqargan konfuzion matritsa va umumiy natijalar grafiklarini ko'rib chiqamiz.


In [ ]:
image_files = [
    ('Konfuzion matritsa', results_dir / 'confusion_matrix.png'),
    ('Normallashtirilgan konfuzion matritsa', results_dir / 'confusion_matrix_normalized.png'),
    ("Validatsiya ko'rsatkichlari", results_dir / 'results.png')
]

for title, path in image_files:
    if path.exists():
        display(Image(filename=str(path)))
    else:
        print(f"{title}: {path} topilmadi")


### Mashg'ulot paytidagi namunaviy batchlar

Quyidagi rasmlar modelga uzatilgan batchlarni va ularning ustidagi tayyorlangan annotatsiyalarni ko'rsatadi.


In [ ]:
sample_batches = sorted(results_dir.glob('train_batch*.jpg'))[:3]
for path in sample_batches:
    display(Image(filename=str(path)))


## Mashg'ulot konfiguratsiyasi

`args.yaml` faylida trening uchun ishlatilgan parametrlar ko'rsatilgan.


In [ ]:
args_path = results_dir / 'args.yaml'
with open(args_path, 'r') as f:
    args = yaml.safe_load(f)

config_series = pd.Series(args, name='Qiymat')
config_series


## Qisqa xulosa

- Aniqlik va qayta chaqirish o'rtacha darajada bo'lsa-da, mAP@50-95 ko'rsatkichlari ma'lumotlar to'plamida hali katta yaxshilash imkoniyati borligini ko'rsatadi.
- Validatsiya yo'qotishlari barqarorlashganidan so'ng ham mAP@50-95 ko'rsatkichlarida tebranishlar kuzatilmoqda, bu esa yanada ko'proq epoch yoki balanslangan dataset talab qilishi mumkinligini anglatadi.
- Konfuzion matritsa normal va benign sinflarini ajratishda model nisbatan yaxshi, ammo malignant sinfi uchun qo'shimcha ma'lumot yoki data augmentatsiyasi zarur.
- Trening konfiguratsiyasi (imgsz=1280, YOLO11L) yuqori aniqlik uchun mos, ammo inferens tezligi va resurs talablari hisobga olinishi lozim.

Keyingi qadamlarda inferens pipelineidagi thresholdlarni optimallashtirish, qaysi sinfda xatoliklar ko'p sodir bo'layotganini chuqur tahlil qilish va klinik ekspertlardan qayta belgilash olish tavsiya etiladi.
